This code aggregates the overall customer behavior profile for each month created from the notebook: Customer_Behavior_Profile_Month.ipynb

In [3]:
# Loading basic needed libraries
import pandas as pd
import numpy as np
import gc
from functools import reduce
import datetime as dt
from datetime import date

# Loading libraries for S3 bucket connection
import boto3
import io
from io import StringIO,BytesIO, TextIOWrapper
import gzip

client = boto3.client('s3') 
resource = boto3.resource('s3')

import plotly.express as px

#### Creating full customer profile

In [4]:
# Reading customer overall profiles from S3
df_oct = pd.read_csv('s3://myaws-capstone-bucket/data/2019-Oct/customer_profile.csv')
df_nov = pd.read_csv('s3://myaws-capstone-bucket/data/2019-Nov/customer_profile.csv')
df_dec = pd.read_csv('s3://myaws-capstone-bucket/data/2019-Dec/customer_profile.csv')
df_jan = pd.read_csv('s3://myaws-capstone-bucket/data/2020-Jan/customer_profile.csv')
df_feb = pd.read_csv('s3://myaws-capstone-bucket/data/2020-Feb/customer_profile.csv')
df_mar = pd.read_csv('s3://myaws-capstone-bucket/data/2020-Mar/customer_profile.csv')
df_apr = pd.read_csv('s3://myaws-capstone-bucket/data/2020-Apr/customer_profile.csv')

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (12,13) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
# Concatanating dfs together to create one full customer profile for the 7 months
data_frames = [df_oct, df_nov, df_dec, df_jan, df_feb, df_mar, df_apr]
cust_profile = pd.concat(data_frames)
cust_profile.nunique()

user_id              13699114
view_count               1795
cart_count                348
purchase_count            262
session_count             541
spent                  365873
min_spent               57617
max_spent               61507
median_spent           168444
avg_spent              454231
min_activity_date         213
max_activity_date         213
min_view_date             213
max_view_date             213
min_cart_date             213
max_cart_date             213
min_purchase_date         211
max_purchase_date         211
month                       7
dtype: int64

In [6]:
# Rank column based on month customer shopped
cust_profile.loc[(cust_profile['month'] == '2019-Oct'),'month_rank'] = 7
cust_profile.loc[(cust_profile['month'] == '2019-Nov'),'month_rank'] = 6
cust_profile.loc[(cust_profile['month'] == '2019-Dec'),'month_rank'] = 5
cust_profile.loc[(cust_profile['month'] == '2020-Jan'),'month_rank'] = 4
cust_profile.loc[(cust_profile['month'] == '2020-Feb'),'month_rank'] = 3
cust_profile.loc[(cust_profile['month'] == '2020-Mar'),'month_rank'] = 2
cust_profile.loc[(cust_profile['month'] == '2020-Apr'),'month_rank'] = 1

In [7]:
cust_profile.nunique()

user_id              13699114
view_count               1795
cart_count                348
purchase_count            262
session_count             541
spent                  365873
min_spent               57617
max_spent               61507
median_spent           168444
avg_spent              454231
min_activity_date         213
max_activity_date         213
min_view_date             213
max_view_date             213
min_cart_date             213
max_cart_date             213
min_purchase_date         211
max_purchase_date         211
month                       7
month_rank                  7
dtype: int64

In [8]:
# Saving results with month data added
cust_profile.to_csv('s3://myaws-capstone-bucket/data/cust_profile_full_month_agg.csv',index=False)

In [9]:
# Transforming date columns to datetime type
cust_profile["max_activity_date"] = cust_profile["max_activity_date"].astype('datetime64[ns]')
cust_profile["min_activity_date"] = cust_profile["min_activity_date"].astype('datetime64[ns]')

cust_profile["min_view_date"] = cust_profile["min_view_date"].astype('datetime64[ns]')
cust_profile["max_view_date"] = cust_profile["max_view_date"].astype('datetime64[ns]')

cust_profile["min_cart_date"] = cust_profile["min_cart_date"].astype('datetime64[ns]')
cust_profile["max_cart_date"] = cust_profile["max_cart_date"].astype('datetime64[ns]')

cust_profile["max_purchase_date"] = cust_profile["max_purchase_date"].astype('datetime64[ns]')
cust_profile["min_purchase_date"] = cust_profile["min_purchase_date"].astype('datetime64[ns]')

In [11]:
# Grabbing max dates based on month rank
max_activity_dates = cust_profile.sort_values('max_activity_date',ascending = False).groupby('user_id').head(1)
max_activity_dates = max_activity_dates[['user_id','max_activity_date']]

max_view_dates = cust_profile.sort_values('max_view_date',ascending = False).groupby('user_id').head(1)
max_view_dates = max_view_dates[['user_id','max_view_date']]

max_cart_dates = cust_profile.sort_values('max_cart_date',ascending = False).groupby('user_id').head(1)
max_cart_dates = max_cart_dates[['user_id','max_cart_date']]

max_purchase_dates = cust_profile.sort_values('max_purchase_date',ascending = False).groupby('user_id').head(1)
max_purchase_dates = max_purchase_dates[['user_id','max_purchase_date']]

# merging_max_dates
data_frames = [max_activity_dates,max_view_dates, max_cart_dates, max_purchase_dates]
max_dates = reduce(lambda  left,right: pd.merge(left,right,on=['user_id'],how='outer'), data_frames)

In [12]:
# Grabbing min dates based on month rank
min_activity_dates = cust_profile.sort_values('min_activity_date',ascending = True).groupby('user_id').head(1)
min_activity_dates = min_activity_dates[['user_id','min_activity_date']]

min_view_dates = cust_profile.sort_values('min_view_date',ascending = True).groupby('user_id').head(1)
min_view_dates = min_view_dates[['user_id','min_view_date']]

min_cart_dates = cust_profile.sort_values('min_cart_date',ascending = True).groupby('user_id').head(1)
min_cart_dates = min_cart_dates[['user_id','min_cart_date']]

min_purchase_dates = cust_profile.sort_values('min_purchase_date',ascending = True).groupby('user_id').head(1)
min_purchase_dates = min_purchase_dates[['user_id','min_purchase_date']]

# merging_min_dates
data_frames = [min_activity_dates,min_view_dates, min_cart_dates, min_purchase_dates]
min_dates = reduce(lambda  left,right: pd.merge(left,right,on=['user_id'],how='outer'), data_frames)

In [13]:
# Obtaining max possible date from data 
column = df_apr["max_activity_date"].astype('datetime64[ns]')
max_date = column.max() 
# Transforming date columns to datetime format
max_dates[['max_activity_date','max_view_date','max_cart_date', 'max_purchase_date']] = max_dates[['max_activity_date','max_view_date','max_cart_date', 'max_purchase_date']].astype('datetime64[ns]')
min_dates[['min_activity_date','min_view_date','min_cart_date', 'min_purchase_date']] = min_dates[['min_activity_date','min_view_date','min_cart_date', 'min_purchase_date']].astype('datetime64[ns]')

customer_dates = pd.merge(max_dates, min_dates, on=["user_id"], how='inner')

In [14]:
customer_dates.head()

,user_id,max_activity_date,max_view_date,max_cart_date,max_purchase_date,min_activity_date,min_view_date,min_cart_date,min_purchase_date
0,649742534,2020-04-30,NaT,NaT,2020-04-30,2020-04-30,NaT,NaT,2020-04-30
1,646652794,2020-04-30,2020-04-30,2020-04-30,2020-04-24,2020-04-24,2020-04-24,2020-04-24,2020-04-24
2,640994963,2020-04-30,2020-04-30,2020-04-25,NaT,2020-04-13,2020-04-13,2020-04-25,NaT
3,646651883,2020-04-30,2020-04-30,2020-04-24,NaT,2020-04-24,2020-04-24,2020-04-24,NaT
4,556979353,2020-04-30,2020-04-30,2020-03-17,NaT,2019-10-05,2019-10-05,2019-12-03,NaT


In [16]:
# Calculating days since each event and customer activity
customer_dates['cust_retailer_age'] = (max_date - customer_dates['min_activity_date']).dt.days
customer_dates['days_since_last_activity'] = (max_date - customer_dates['max_activity_date']).dt.days

customer_dates['first_view_age'] = (max_date - customer_dates['min_view_date']).dt.days
customer_dates['days_since_last_view'] = (max_date - customer_dates['max_view_date']).dt.days

customer_dates['first_cart_age'] = (max_date - customer_dates['min_cart_date']).dt.days
customer_dates['days_since_last_cart'] = (max_date - customer_dates['max_cart_date']).dt.days


customer_dates['first_purchase_age'] = (max_date - customer_dates['min_purchase_date']).dt.days
customer_dates['days_since_last_purchase'] = (max_date - customer_dates['max_purchase_date']).dt.days

customer_dates[['cust_retailer_age',
       'days_since_last_activity', 'first_view_age', 'days_since_last_view',
       'first_cart_age', 'days_since_last_cart', 'first_purchase_age',
       'days_since_last_purchase']] = customer_dates[['cust_retailer_age',
       'days_since_last_activity', 'first_view_age', 'days_since_last_view',
       'first_cart_age', 'days_since_last_cart', 'first_purchase_age',
       'days_since_last_purchase']].fillna(value=9999)# Replacing nan with 9999 to symbolize they have not done said action

customer_dates.head()

,user_id,max_activity_date,max_view_date,max_cart_date,max_purchase_date,min_activity_date,min_view_date,min_cart_date,min_purchase_date,cust_retailer_age,days_since_last_activity,first_view_age,days_since_last_view,first_cart_age,days_since_last_cart,first_purchase_age,days_since_last_purchase
0,649742534,2020-04-30,NaT,NaT,2020-04-30,2020-04-30,NaT,NaT,2020-04-30,0,0,9999.0,9999.0,9999.0,9999.0,0.0,0.0
1,646652794,2020-04-30,2020-04-30,2020-04-30,2020-04-24,2020-04-24,2020-04-24,2020-04-24,2020-04-24,6,0,6.0,0.0,6.0,0.0,6.0,6.0
2,640994963,2020-04-30,2020-04-30,2020-04-25,NaT,2020-04-13,2020-04-13,2020-04-25,NaT,17,0,17.0,0.0,5.0,5.0,9999.0,9999.0
3,646651883,2020-04-30,2020-04-30,2020-04-24,NaT,2020-04-24,2020-04-24,2020-04-24,NaT,6,0,6.0,0.0,6.0,6.0,9999.0,9999.0
4,556979353,2020-04-30,2020-04-30,2020-03-17,NaT,2019-10-05,2019-10-05,2019-12-03,NaT,208,0,208.0,0.0,149.0,44.0,9999.0,9999.0


In [17]:
# Grabbing min and max spent
max_spent = cust_profile.groupby(['user_id']).max_spent.agg(max_spent=np.max).reset_index()
min_spent = cust_profile.groupby(['user_id']).min_spent.agg(min_spent=np.min).reset_index()

customer_spent = pd.merge(min_spent, max_spent, on=["user_id"], how='inner')
customer_spent.head()

,user_id,min_spent,max_spent
0,12511517,0.0,0.0
1,22165363,0.0,0.0
2,29515875,0.0,0.0
3,29830839,0.0,0.0
4,29990697,0.0,0.0


In [18]:
# Merging spent and date dfs
customer_agg = pd.merge(customer_spent, customer_dates[['user_id','cust_retailer_age',
       'days_since_last_activity', 'first_view_age', 'days_since_last_view',
       'first_cart_age', 'days_since_last_cart', 'first_purchase_age',
       'days_since_last_purchase']], on=["user_id"], how='inner')
customer_agg.nunique()

user_id                     13699114
min_spent                      39053
max_spent                      57826
cust_retailer_age                213
days_since_last_activity         213
first_view_age                   214
days_since_last_view             214
first_cart_age                   214
days_since_last_cart             214
first_purchase_age               212
days_since_last_purchase         212
dtype: int64

In [19]:
# Aggregating numeric columns from cust_profile
customer_views = pd.DataFrame(cust_profile.groupby(['user_id'])['view_count'].agg('sum')).reset_index()
customer_views.columns = ['user_id','total_view']# Renaming columns
customer_carts = pd.DataFrame(cust_profile.groupby(['user_id'])['cart_count'].agg('sum')).reset_index()
customer_carts.columns = ['user_id','total_cart_add']# Renaming columns
customer_purchases = pd.DataFrame(cust_profile.groupby(['user_id'])['purchase_count'].agg('sum')).reset_index()
customer_purchases.columns = ['user_id','total_purchases']# Renaming columns
customer_sessions = pd.DataFrame(cust_profile.groupby(['user_id'])['session_count'].agg('sum')).reset_index()
customer_sessions.columns = ['user_id','total_sessions']# Renaming columns
customer_spent = pd.DataFrame(cust_profile.groupby(['user_id'])['spent'].agg('sum')).reset_index()
customer_spent.columns = ['user_id','total_spent']# Renaming columns

# Merging event counts dfs together
data_frames = [customer_views, customer_carts, customer_purchases, customer_sessions, customer_spent]
customer_profile = reduce(lambda  left,right: pd.merge(left,right,on=['user_id'],how='outer'), data_frames)
customer_profile.nunique()

user_id            13699114
total_view             2722
total_cart_add          490
total_purchases         367
total_sessions          551
total_spent          419400
dtype: int64

In [20]:
customer_profile = pd.merge(customer_profile, customer_agg, on=["user_id"], how='inner')
customer_profile.head()

,user_id,total_view,total_cart_add,total_purchases,total_sessions,total_spent,min_spent,max_spent,cust_retailer_age,days_since_last_activity,first_view_age,days_since_last_view,first_cart_age,days_since_last_cart,first_purchase_age,days_since_last_purchase
0,12511517,1.0,0.0,0.0,1.0,0.0,0.0,0.0,53,53,53.0,53.0,9999.0,9999.0,9999.0,9999.0
1,22165363,10.0,0.0,0.0,2.0,0.0,0.0,0.0,78,48,78.0,48.0,9999.0,9999.0,9999.0,9999.0
2,29515875,18.0,0.0,0.0,6.0,0.0,0.0,0.0,172,7,172.0,7.0,9999.0,9999.0,9999.0,9999.0
3,29830839,1.0,0.0,0.0,1.0,0.0,0.0,0.0,100,100,100.0,100.0,9999.0,9999.0,9999.0,9999.0
4,29990697,12.0,0.0,0.0,12.0,0.0,0.0,0.0,78,1,78.0,1.0,9999.0,9999.0,9999.0,9999.0


In [21]:
customer_profile.nunique()

user_id                     13699114
total_view                      2722
total_cart_add                   490
total_purchases                  367
total_sessions                   551
total_spent                   419400
min_spent                      39053
max_spent                      57826
cust_retailer_age                213
days_since_last_activity         213
first_view_age                   214
days_since_last_view             214
first_cart_age                   214
days_since_last_cart             214
first_purchase_age               212
days_since_last_purchase         212
dtype: int64

In [22]:
customer_profile.shape

(13699114, 16)

In [23]:
# Saving results in S3
customer_profile.to_csv('s3://myaws-capstone-bucket/data/customer_profile_full.csv',index=False)

#### Analyzing customer behavior

In [24]:
pd.options.display.float_format = '{:.2f}'.format
customer_profile.describe()

,user_id,total_view,total_cart_add,total_purchases,total_sessions,total_spent,min_spent,max_spent,cust_retailer_age,days_since_last_activity,first_view_age,days_since_last_view,first_cart_age,days_since_last_cart,first_purchase_age,days_since_last_purchase
count,13699114.00,13699114.00,13699114.00,13699114.00,13699114.00,13699114.00,13699114.00,13699114.00,13699114.00,13699114.00,13699114.00,13699114.00,13699114.00,13699114.00,13699114.00,13699114.00
mean,581718124.97,20.79,1.14,0.42,6.53,138.94,13.41,50.10,111.89,77.38,115.77,81.35,7723.15,7717.95,8686.82,8684.22
std,40233166.81,88.11,5.33,3.04,19.67,1427.03,87.38,188.39,61.87,58.58,205.28,204.97,4164.46,4173.96,3355.43,3362.09
min,12511517.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,550709450.25,2.00,0.00,0.00,1.00,0.00,0.00,0.00,61.00,22.00,61.00,22.00,9999.00,9999.00,9999.00,9999.00
50%,585832329.00,4.00,0.00,0.00,3.00,0.00,0.00,0.00,116.00,69.00,117.00,69.00,9999.00,9999.00,9999.00,9999.00
75%,614615740.00,15.00,0.00,0.00,7.00,0.00,0.00,0.00,165.00,122.00,165.00,122.00,9999.00,9999.00,9999.00,9999.00
max,649775983.00,145465.00,2186.00,1975.00,29140.00,790098.29,2574.07,2574.07,212.00,212.00,9999.00,9999.00,9999.00,9999.00,9999.00,9999.00


In [25]:
none_purchase_custs = customer_profile.loc[(customer_profile['total_spent'] == 0)]
none_purchase_custs.nunique()

user_id                     11881941
total_view                      1935
total_cart_add                   176
total_purchases                    1
total_sessions                   527
total_spent                        1
min_spent                          1
max_spent                          1
cust_retailer_age                213
days_since_last_activity         213
first_view_age                   214
days_since_last_view             214
first_cart_age                   214
days_since_last_cart             214
first_purchase_age                 1
days_since_last_purchase           1
dtype: int64

In [26]:
none_purchase_custs.describe()

,user_id,total_view,total_cart_add,total_purchases,total_sessions,total_spent,min_spent,max_spent,cust_retailer_age,days_since_last_activity,first_view_age,days_since_last_view,first_cart_age,days_since_last_cart,first_purchase_age,days_since_last_purchase
count,11881941.00,11881941.00,11881941.00,11881941.00,11881941.00,11881941.00,11881941.00,11881941.00,11881941.00,11881941.00,11881941.00,11881941.00,11881941.00,11881941.00,11881941.00,11881941.00
mean,584116005.85,13.21,0.28,0.00,5.25,0.00,0.00,0.00,107.82,79.92,109.86,82.01,8834.75,8833.64,9999.00,9999.00
std,39630198.78,73.70,1.47,0.00,19.91,0.00,0.00,0.00,60.84,58.19,154.50,153.85,3189.88,3192.94,0.00,0.00
min,12511517.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,9999.00,9999.00
25%,554937574.00,1.00,0.00,0.00,1.00,0.00,0.00,0.00,58.00,26.00,58.00,26.00,9999.00,9999.00,9999.00,9999.00
50%,588711715.00,3.00,0.00,0.00,2.00,0.00,0.00,0.00,111.00,72.00,111.00,72.00,9999.00,9999.00,9999.00,9999.00
75%,616053420.00,10.00,0.00,0.00,5.00,0.00,0.00,0.00,158.00,124.00,158.00,124.00,9999.00,9999.00,9999.00,9999.00
max,649775983.00,145465.00,807.00,0.00,29140.00,0.00,0.00,0.00,212.00,212.00,9999.00,9999.00,9999.00,9999.00,9999.00,9999.00


In [27]:
11881941/13699114

0.8673510564259849

In [28]:
purchase_custs = customer_profile.loc[(customer_profile['total_spent'] > 0)]
purchase_custs.nunique()

user_id                     1817173
total_view                     2424
total_cart_add                  484
total_purchases                 366
total_sessions                  306
total_spent                  419399
min_spent                     39053
max_spent                     57825
cust_retailer_age               213
days_since_last_activity        213
first_view_age                  214
days_since_last_view            214
first_cart_age                  214
days_since_last_cart            214
first_purchase_age              211
days_since_last_purchase        211
dtype: int64

In [29]:
purchase_custs.describe()

,user_id,total_view,total_cart_add,total_purchases,total_sessions,total_spent,min_spent,max_spent,cust_retailer_age,days_since_last_activity,first_view_age,days_since_last_view,first_cart_age,days_since_last_cart,first_purchase_age,days_since_last_purchase
count,1817173.00,1817173.00,1817173.00,1817173.00,1817173.00,1817173.00,1817173.00,1817173.00,1817173.00,1817173.00,1817173.00,1817173.00,1817173.00,1817173.00,1817173.00,1817173.00
mean,566039111.99,70.34,6.75,3.14,14.94,1047.44,101.11,377.66,138.50,60.75,154.43,77.02,454.69,422.85,106.90,87.25
std,40622062.90,142.05,12.79,7.81,15.54,3794.79,220.67,379.26,61.93,58.39,399.83,402.40,1816.12,1822.08,62.78,60.75
min,101875240.00,0.00,0.00,1.00,1.00,0.42,0.00,0.42,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,526974250.00,10.00,2.00,1.00,5.00,158.94,0.00,132.56,93.00,9.00,94.00,9.00,59.00,25.00,53.00,33.00
50%,561943831.00,30.00,4.00,2.00,10.00,348.92,0.00,231.64,155.00,44.00,155.00,44.00,123.00,72.00,113.00,79.00
75%,598362706.00,79.00,7.00,3.00,20.00,900.92,130.25,476.18,194.00,101.00,194.00,103.00,165.00,129.00,164.00,135.00
max,649772024.00,57349.00,2186.00,1975.00,5788.00,790098.29,2574.07,2574.07,212.00,212.00,9999.00,9999.00,9999.00,9999.00,212.00,212.00


In [1]:
1817173/13699114*100

13.264894357401507

In [31]:
# Saving results into S3
purchase_custs.to_csv('s3://myaws-capstone-bucket/data/customers_of_focus.csv',index=False)